In [19]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure,show
import matplotlib.pyplot as plt
from pyts.image import RecurrencePlot
import tensorflow
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,InputLayer,Dropout
from keras import Sequential
from keras.optimizers import Adam
import re
from sklearn.model_selection import train_test_split
from sklearn import metrics
import kerastuner as kt

In [20]:
data_attack_free,data_spoofing_attack,data_fuzzy_attack,data_DoS_attack=[],[],[],[]
with open("D:\IIT-Delhi\Semester-2\SIL765-Networks & System Security\Project\Implementation\DataSets\Attack_free_dataset.txt") as f_a:
    data_attack_free.extend([re.sub(r"Timestamp:\s+(\d+\.\d+)\s+ID:\s+([\da-f]{4})\s+\d+\s+DLC:\s+\d+(\s+[\da-f]{2})*", r"\1@\2", lines).strip("\n").split("@") for lines in f_a.readlines()])

with open("D:\IIT-Delhi\Semester-2\SIL765-Networks & System Security\Project\Implementation\DataSets\Impersonation_attack_dataset.txt") as f_b:
    data_spoofing_attack.extend([re.sub(r"Timestamp:\s+(\d+\.\d+)\s+ID:\s+([\da-f]{4})\s+\d+\s+DLC:\s+\d+(\s+[\da-f]{2})*", r"\1@\2", lines).strip("\n").split("@") for lines in f_b.readlines()])

with open("D:\IIT-Delhi\Semester-2\SIL765-Networks & System Security\Project\Implementation\DataSets\Fuzzy_attack_dataset.txt") as f_c:
    data_fuzzy_attack.extend([re.sub(r"Timestamp:\s+(\d+\.\d+)\s+ID:\s+([\da-f]{4})\s+\d+\s+DLC:\s+\d+(\s+[\da-f]{2})*", r"\1@\2", lines).strip("\n").split("@") for lines in f_c.readlines()])

with open("D:\IIT-Delhi\Semester-2\SIL765-Networks & System Security\Project\Implementation\DataSets\DoS_attack_dataset.txt") as f_d:
    data_DoS_attack.extend([re.sub(r"Timestamp:\s+(\d+\.\d+)\s+ID:\s+([\da-f]{4})\s+\d+\s+DLC:\s+\d+(\s+[\da-f]{2})*", r"\1@\2", lines).strip("\n").split("@") for lines in f_d.readlines()])
a_id_attack_free,a_id_spoofing_attack,a_id_fuzzy_attack,a_id_DoS_attack=[],[],[],[]
for i in range(len(data_attack_free)):
    a_id_attack_free.append(str(data_attack_free[i][1]))
for i in range(len(data_spoofing_attack)):
    a_id_spoofing_attack.append(str(data_spoofing_attack[i][1]))
for i in range(len(data_fuzzy_attack)):
    a_id_fuzzy_attack.append(str(data_fuzzy_attack[i][1]))
for i in range(len(data_DoS_attack)):
    a_id_DoS_attack.append(str(data_DoS_attack[i][1]))
a1,a2,a3,a4=[],[],[],[]
for hex_num in a_id_attack_free:
    dec_value = int(hex_num, 16)
    a1.append(dec_value)
for hex_num in a_id_spoofing_attack:
    dec_value = int(hex_num, 16)
    a2.append(dec_value)
for hex_num in a_id_fuzzy_attack:
    dec_value = int(hex_num, 16)
    a3.append(dec_value)
for hex_num in a_id_DoS_attack:
    dec_value = int(hex_num, 16)
    a4.append(dec_value)
range1=int(np.floor(len(a1)/(128*128)))
range2=int(np.floor(len(a2)/(128*128)))
range3=int(np.floor(len(a3)/(128*128)))
range4=int(np.floor(len(a4)/(128*128)))
storage=np.array([])
rp = RecurrencePlot(dimension=1,threshold='point',percentage=40)
temp1=[]
count=0
for i in range(range1):
    temp=np.array(a1[(i*(128**2)):((i+1)*(128**2))])
    temp=temp.reshape(128,128)
    plot = rp.fit_transform(temp)
    plot=np.array(plot[0])
    plot=plot.astype(int)
    storage=np.append(values=plot,arr=storage)
    storage=storage.reshape(count+1,128,128,1)
    count=count+1
for i in range(range2):
    temp=np.array(a2[(i*(128**2)):((i+1)*(128**2))])
    temp=temp.reshape(128,128)
    plot = rp.fit_transform(temp)
    plot=np.array(plot[0])
    plot=plot.astype(int)
    storage=np.append(values=plot,arr=storage)
    storage=storage.reshape(count+1,128,128,1)
    count=count+1
for i in range(range3):
    temp=np.array(a3[(i*(128**2)):((i+1)*(128**2))])
    temp=temp.reshape(128,128)
    plot = rp.fit_transform(temp)
    plot=np.array(plot[0])
    plot=plot.astype(int)
    storage=np.append(values=plot,arr=storage)
    storage=storage.reshape(count+1,128,128,1)
    count=count+1
for i in range(range4):
    temp=np.array(a4[(i*(128**2)):((i+1)*(128**2))])
    temp=temp.reshape(128,128)
    plot = rp.fit_transform(temp)
    plot=np.array(plot[0])
    plot=plot.astype(int)
    storage=np.append(values=plot,arr=storage)
    storage=storage.reshape(count+1,128,128,1)
    count=count+1

In [31]:
X=storage
y=np.array([])
y=np.append(values=[0]*range1,arr=y)
y=np.append(values=[1]*(range2+range3+range4),arr=y)
y=y.astype(int)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [22]:
# model=Sequential()
# model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",activation="relu",input_shape=(128,128,1)))
# model.add(MaxPooling2D(pool_size=(2,2),padding="valid",strides=2))
# model.add(Conv2D(filters=256,kernel_size=(3,3),padding="same",activation="relu"))
# model.add(MaxPooling2D(pool_size=(2,2),padding="valid",strides=2))
# model.add(Dropout(0.9))
# model.add(Flatten())
# model.add(Dense(units=16,activation="relu"))
# model.add(Dense(units=1,activation="sigmoid"))
# adam = Adam(learning_rate=0.0001)
# model.compile(optimizer=adam,loss="binary_crossentropy",metrics=["accuracy"])

In [23]:
# history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))
# loss, accuracy = model.evaluate(X_test, y_test)

In [27]:
def build_model(hp):
    model=Sequential()
    model.add(Conv2D(filters=64,
                     kernel_size=(3,3),
                     padding="same",activation="relu",
                     input_shape=(128,128,1)))
    model.add(MaxPooling2D(pool_size=(2,2),
                           padding="valid",
                           strides=2))
    model.add(Conv2D(filters=256,
                     kernel_size=(3,3),
                     padding="same",
                     activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),
                           padding="valid",
                           strides=2))
    model.add(Dropout(0.9))
    model.add(Flatten())
    model.add(Dense(units=16,                   
                    activation="relu"))
    for i in range(hp.Int('num_layers',
                          min_value=8,
                          max_value=16)):
        model.add(Dense(hp.Int('units'+str(i),
                               min_value=8,
                               max_value=16),
                               activation=hp.Choice('activation'+str(i),
                                                    values=['relu','tanh','sigmoid'])))
        model.add(Dropout(hp.Choice('droput'+str(i),
                                    values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(Dense(units=1,
                    activation="sigmoid"))
    optimizer=hp.Choice('optimizer',
                        values=['adam','sgd','rmsdrop','adamw','adadelta',
                                'adagrad','adamax','adafactor','nadam','ftrl'])
    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",metrics=["accuracy"])
    return model
tuner=kt.RandomSearch(build_model,objective='accuracy',
                      max_trials=3,directory='mydir',project_name="final")
tuner.search(X_train,y_train,epochs=5,validation_data=(X_val,y_val))

Trial 3 Complete [00h 01m 04s]
accuracy: 0.572139322757721

Best accuracy So Far: 0.572139322757721
Total elapsed time: 00h 03m 14s
INFO:tensorflow:Oracle triggered exit


In [28]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 8,
 'units0': 10,
 'activation0': 'relu',
 'droput0': 0.3,
 'units1': 8,
 'activation1': 'tanh',
 'droput1': 0.2,
 'units2': 16,
 'activation2': 'tanh',
 'droput2': 0.4,
 'units3': 8,
 'activation3': 'tanh',
 'droput3': 0.1,
 'units4': 9,
 'activation4': 'sigmoid',
 'droput4': 0.9,
 'units5': 16,
 'activation5': 'sigmoid',
 'droput5': 0.7,
 'units6': 8,
 'activation6': 'tanh',
 'droput6': 0.2,
 'units7': 10,
 'activation7': 'sigmoid',
 'droput7': 0.6,
 'optimizer': 'adam',
 'units8': 11,
 'activation8': 'tanh',
 'droput8': 0.4,
 'units9': 16,
 'activation9': 'sigmoid',
 'droput9': 0.2,
 'units10': 14,
 'activation10': 'relu',
 'droput10': 0.3,
 'units11': 11,
 'activation11': 'tanh',
 'droput11': 0.1,
 'units12': 14,
 'activation12': 'sigmoid',
 'droput12': 0.4}

In [33]:
model=tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 256)       147712    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 256)      0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 32, 32, 256)       0         
                                                                 
 flatten (Flatten)           (None, 262144)            0

In [32]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))
loss, accuracy = model.evaluate(X_test, y_test)

Epoch 1/10
7/7 [==============================] - 21s 3s/step - loss: 0.7027 - accuracy: 0.5174 - val_loss: 0.6939 - val_accuracy: 0.4783
Epoch 2/10
7/7 [==============================] - 19s 3s/step - loss: 0.7229 - accuracy: 0.4876 - val_loss: 0.6939 - val_accuracy: 0.4783
Epoch 3/10
7/7 [==============================] - 23s 3s/step - loss: 0.7488 - accuracy: 0.4577 - val_loss: 0.6939 - val_accuracy: 0.4783
Epoch 4/10
7/7 [==============================] - 29s 4s/step - loss: 0.7198 - accuracy: 0.5025 - val_loss: 0.6939 - val_accuracy: 0.4783
Epoch 5/10
7/7 [==============================] - 29s 4s/step - loss: 0.7000 - accuracy: 0.5572 - val_loss: 0.6940 - val_accuracy: 0.4783
Epoch 6/10
7/7 [==============================] - 25s 4s/step - loss: 0.7201 - accuracy: 0.4726 - val_loss: 0.6943 - val_accuracy: 0.4783
Epoch 7/10
7/7 [==============================] - 26s 4s/step - loss: 0.7233 - accuracy: 0.4975 - val_loss: 0.6944 - val_accuracy: 0.4783
Epoch 8/10
7/7 [==================